## Decorador (*Decorator*)

__Decorador__ es un patron de tipo Estructural. Nos permite "envolver" un
objeto, método o función, que realiza una determinada función, con otro objeto,
método o función que copia su interfaz; es decir, que para el resto del mundo se
comporta y se puede interactuar con el exactamente igual que si fuera el
original. De esta forma se puede modificar el comportamiento, normalmente para
añadir funcionalidad.

La razón del nombre de este componente viene de uno de sus primeros ejemplos de
uso, en los sistemas de ventanas. En estos sistemas, el contenido visual de una
aplicación no tiene bordes, ni botones, ni controles como barras de
desplazamiento, etc... Es solo un cuadrado con el contenido de la aplicación. Se
aplicaban una función (se decoraba) al componente, de forma que, por ejemplo,
ante una operación de dibujar la ventana, digamos `redraw`, el decorador pintaba
los bordes, los botones, etc. y luego llamaba a la operación `redraw` de la
aplicación, que pintaba los contenidos.

Eso permitia cambiar el sistema de ventanas para tener estilos totalmente
diferentes, solo cambiando el decorador que "envuelve" a la aplicación. Incluso
si eliminabas el decorador, el sistema seguía funcionando (Sin bordes, claro)
porque al sistema de ventanas solo le interesaba que los componentes tengan un
método `redraw` (la interfaz es la misma).

Las razones para usar este patrón son normalmente dos:

- Mejorar la respuesta de un componente a otro componente

- Proporcionar multiples comportamientos opcionales

Se utiliza a menudo la segunda opción como una alternativa a la herencia
múltiple. Se puede crear un objeto básico, y luego "envolverlo" con un
decorador. Como la interfaz del decorador es igual que la del objeto base,
podemos usar diferentes decoradores para el mismo objeto o incluso anidar
decoradores, es decir, decorando un objeto ya decorado.


## Un ejemplo de decorador.

Usaremos un ejemplo de decorador a la vez que vemos algo de programación de redes. En este
caso usaremos un *socket* TCP. EL método `send` de los *sockets* acepta como parámetro
una secuencia de bytes y los envia por la conexión establecida hacie el otro extremo. Hay muchas
librerías de sockets pero python viene con una ya incorporada en la libreria estándar.

Vamos a crear en primer lugar un servidor que espere a que algun otro proceso se conecte; en
ese momento pregunta un texto el usuario, y devuelve ese texto como respuesta.

In [ ]:
# %load socket-server.py
#!/usr/bin/env python

import socket

def respond(client, addr):
    response = input("Enter a value: ")
    message = f"Hello {addr} this is your response: {response}"
    client.send(message.encode('utf8'))
    client.close()
    return response == 'exit'

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(('localhost', 8401))
server.listen(1)
try:
    while True:
        client, addr = server.accept()
        must_exit = respond(client, addr)
        if must_exit:
            break
finally:
    server.close()

Ahora, antes de escribir el cliente, un aviso: para poder ejecutar la cliente y el servicor, estos
tienen que ejecutarse como procesos distintos, asi que hay que ejecutar el servidor en un notebook, con
su propio kernel, y el cliente en otro. Hay dos notebooks ya preparados para este proposito
[socket-server.ipynb](./socket-server.ipynb) y [socket-client.ipynb](./socket-client.ipynb)

In [16]:
!cat socket-client.py

#!/usr/bin/env python
# coding: utf-8

import socket


client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(('localhost', 8401))
response = client.recv(1024).decode('utf-8')
print(f"Received: {response}".format(client.recv(1024)))
client.close()

Para usar estos programas:
    
❶ Arrancar el programa `socket-server.py`, ya sea en una terminal o abriendo en
una pestaña nueva del navegador con [`socket-server.ipynb`](./socket-server.ipynb).

❷ Arrancar el programa `socket-client.py`, ya sea en una terminal o abriendo en
una pestaña nueva del navegador con [`socket-client.ipynb`](./socket-client.ipynb).

❸ En la pestaña o terminal del servidor veremos que se pregunta al usuario
por el texto de la respuesta; escribir algo y pulsar enter.

❹ En la pestaña / terminal del cliente, veremos que hemos recibido una
respuesta con el texto que metimos.

Puedes ejecutar el cliente las veces que quieras. El cliente enviará los textos
que se introduzcan en el servidor.

Ahora, revisando nuestro codigo servidor, vamos a añadir un par de decoradores para 
el socket. El primero sera un decorador de *Logging*. Este decorador simplemente
imprime en la consola del servidor los datos enviados.

In [ ]:
class LogSocket:

    def __init__(self, socket):
        self.socket = socket

    def send(self, data):
        print("Sending {0} to {1}".format(
            data, self.socket.getpeername()[0]))
        self.socket.send(data)

    def close(self):
        self.socket.close()

El decorador mantiene la misma interfaz que el onjeto socket, asi que, a los
efectos de la función `respond`, lo mismo le da usar un socket normal
o nuestra versión decorada.


### Ejercicio: Modificar el código del servidor para que use nuestra nueva clase decorada

In [17]:
import socket

class LogSocket:

    def __init__(self, socket):
        self.socket = socket

    def send(self, data):
        print("Sending {0} to {1}".format(
            data, self.socket.getpeername()[0]))
        self.socket.send(data)

    def close(self):
        self.socket.close()


def respond(client, addr):
    response = input("Enter a value: ")
    message = f"Hello {addr} this is your response: {response}"
    client.send(message.encode('utf8'))
    client.close()
    return response == 'exit'

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(('localhost', 8401))
server.listen(1)
try:
    while True:
        client, addr = server.accept()
        must_exit = respond(LogSocket(client), addr)
        if must_exit:
            break
finally:
    server.close()
        

Enter a value: Hola, margarita
Sending b"Hello ('127.0.0.1', 62892) this is your response: Hola, margarita" to 127.0.0.1


KeyboardInterrupt: 